In [1]:
import pandas as pd
import pytz
import numpy as np
import os
from sklearn import preprocessing
import re
import matplotlib
matplotlib.use('Qt5Agg')
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def custom_date_parser(date_string):
    return pd.to_datetime(date_string, format='%d-%m-%Y %H:%M:%S')

# Specify the path to the main directory containing folders and files
path = 'D:\\mlinternship\\iitgdata'
folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
df_list = []

# Iterate through each folder
for folder in folders:
    # Construct the full path to the current folder
    folder_path = os.path.join(path, folder)

    # Iterate through files in the current folder
    for filename in os.listdir(folder_path):
        # Check if the file has the '.xlsx' extension
        if filename.endswith('.xlsx'):
            # Construct the full path to the Excel file
            file_path = os.path.join(folder_path, filename)

            # Use the custom date parser function
            df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)

            # Append the dataframe to the list
            df_list.append(df)


C:\Users\jaina\AppData\Local\Temp\ipykernel_34676\3450862427.py:22: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
C:\Users\jaina\AppData\Local\Temp\ipykernel_34676\3450862427.py:22: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_parser=custom_date_parser)
C:\Users\jaina\AppData\Local\Temp\ipykernel_34676\3450862427.py:22: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_excel(file_path, header=3, date_pars

In [3]:
power_df = pd.concat(df_list, ignore_index=True)
power_df.to_csv('power_datacsv.csv')
power_df['Time'] = pd.to_datetime(power_df['Time'])
power_df['Time'] = power_df['Time'].round('min')
#replace all the 'NR' values in MW column to NaN
power_df['MW'] = power_df['MW'].replace('NR', np.nan)
power_df['MW'] = power_df['MW'].replace('nr', np.nan)
full_power_df = power_df.copy()
# drop the rest of the columns that are not required and set index to Time
power_df = power_df[['Time', 'MW']]

In [4]:
power_df

,Time,MW
0,2022-04-16 00:00:00,2.5
1,2022-04-16 01:00:00,2.5
2,2022-04-16 02:00:00,2.5
3,2022-04-16 03:00:00,2.5
4,2022-04-16 04:00:00,2
...,...,...
7438,2022-09-15 19:00:00,5.2
7439,2022-09-15 20:00:00,5
7440,2022-09-15 21:00:00,4.75
7441,2022-09-15 22:00:00,4.65


In [8]:
power_df.dropna(inplace = True)
power_df['MW'] = power_df['MW'].astype(str)
# Remove commas from 'CDH' column and convert to numeric
power_df['MW'] = pd.to_numeric(power_df['MW'].str.replace(',', '.'), errors='coerce')
power_df['Time'] = pd.to_datetime(power_df['Time'])
# Now, 'Time' column is in UTC timezone format
power_df = power_df.sort_values('Time')
power_df

,Time,MW
2933,2022-01-21 02:00:00,2.5
2957,2022-01-22 02:00:00,2.1
3005,2022-01-24 02:00:00,2.4
3101,2022-01-28 02:00:00,2.0
1494,2022-02-01 06:00:00,1.7
...,...,...
5012,2023-03-15 17:00:00,3.9
5013,2023-03-15 18:00:00,4.0
5014,2023-03-15 19:00:00,4.4
5015,2023-03-15 20:00:00,4.2


In [9]:
# read the temperature data csv
temperature_data_csv_path = 'D:\\mlinternship\\iitgdata\\temperaturedata'
filename = 'guwahati_temperature_data.csv'
file = os.path.join(temperature_data_csv_path, filename)
temperature_df = pd.read_csv(file)
#drop the unwanted columns and change column name
temperature_df.rename(columns={'valid': 'Time'}, inplace = True)
temperature_df = temperature_df.rename(columns={'tmpc': 'temperature'})
temperature_df = temperature_df[['Time', 'temperature']]

#changing the time column to a common UTC format
temperature_df['Time'] = pd.to_datetime(temperature_df['Time'])
temperature_df['Time'] = pd.DatetimeIndex(temperature_df['Time']) + timedelta(hours=5,minutes=30)
temperature_df['temperature'] = pd.to_numeric(temperature_df['temperature'], errors='coerce')
temperature_df=temperature_df.dropna()

In [10]:
temperature_df

,Time,temperature
0,2022-01-01 05:30:00,13.0
1,2022-01-01 06:00:00,12.0
2,2022-01-01 06:30:00,12.0
3,2022-01-01 07:00:00,12.0
4,2022-01-01 07:30:00,14.0
...,...,...
16213,2023-01-01 01:00:00,13.0
16214,2023-01-01 01:30:00,13.0
16215,2023-01-01 03:30:00,13.0
16216,2023-01-01 04:30:00,13.0


In [13]:
# joining the two dataframes such that the temperature data is only taken if there exists a reading in the power data dataframe
df = pd.merge(power_df, temperature_df, on='Time', how='left')
DF = df.copy()
#drop all the rows where NaN
df.dropna(inplace=True)
df = df[df['MW'] <= 20]
df = df.sort_values('Time')
df.reset_index(inplace=True, drop = True)
df


,Time,MW,temperature
0,2022-01-24 02:00:00,2.4,16.0
1,2022-01-28 02:00:00,2.0,14.0
2,2022-02-01 06:00:00,1.7,12.0
3,2022-02-01 07:00:00,1.8,13.0
4,2022-02-01 08:00:00,2.3,14.0
...,...,...,...
3791,2022-12-31 17:00:00,2.7,19.0
3792,2022-12-31 19:00:00,2.7,16.0
3793,2022-12-31 21:00:00,2.7,15.0
3794,2022-12-31 22:00:00,2.5,16.0
